In [4]:
import requests

# Set the base URL and endpoint for the NHTSA Safety Ratings API
base_url = "https://api.nhtsa.gov/SafetyRatings"
model_year = 2023
make = "Acura"
model = "rdx"

# Create the full URL with the given model year, make, and model
url = f"{base_url}/modelyear/{model_year}/make/{make}/model/{model}"

# Make a GET request to the NHTSA API
response = requests.get(url)

# Check if the request was successful
if response.status_code == 200:
    # Parse the JSON response
    data = response.json()
    # Display the data
    print(data)
else:
    # If the request failed, print the status code
    print(f"Error: Unable to access data (status code: {response.status_code})")

{'Count': 2, 'Message': 'Results returned successfully', 'Results': [{'VehicleDescription': '2023 Acura RDX SUV AWD', 'VehicleId': 18491}, {'VehicleDescription': '2023 Acura RDX SUV FWD', 'VehicleId': 18490}]}


In [1]:
import requests
import json
import time
import pandas as pd

In [2]:
def get_all_model_years(base_url):
    response = requests.get(base_url)
    if response.status_code == 200:
        data = response.json()
        return [item['ModelYear'] for item in data['Results']]
    return []

def get_all_makes_for_model_year(base_url, model_year):
    response = requests.get(f"{base_url}/modelyear/{model_year}")
    if response.status_code == 200:
        data = response.json()
        return [item['Make'] for item in data['Results']]
    return []

def get_all_models_for_make_and_year(base_url, model_year, make):
    response = requests.get(f"{base_url}/modelyear/{model_year}/make/{make}")
    if response.status_code == 200:
        data = response.json()
        return [item['Model'] for item in data['Results']]
    return []

def get_vehicle_variants(base_url, model_year, make, model):
    response = requests.get(f"{base_url}/modelyear/{model_year}/make/{make}/model/{model}")
    if response.status_code == 200:
        return response.json()['Results']
    return []

def get_safety_ratings_for_variant(base_url, vehicle_id):
    response = requests.get(f"{base_url}/VehicleId/{vehicle_id}")
    if response.status_code == 200:
        return response.json()['Results']
    return []


In [4]:
# Save data to CSV for future use
df.to_csv('safety_ratings_data.csv', index=False)

In [2]:
import requests
import pandas as pd
from datetime import datetime
import time

# Define Functions to Access Data
def get_all_makes_for_model_year(base_url, model_year):
    response = requests.get(f"{base_url}/modelyear/{model_year}")
    if response.status_code == 200:
        data = response.json()
        return [item['Make'] for item in data['Results']]
    return []

def get_all_models_for_make_and_year(base_url, model_year, make):
    response = requests.get(f"{base_url}/modelyear/{model_year}/make/{make}")
    if response.status_code == 200:
        data = response.json()
        return [item['Model'] for item in data['Results']]
    return []

def get_vehicle_variants(base_url, model_year, make, model):
    response = requests.get(f"{base_url}/modelyear/{model_year}/make/{make}/model/{model}")
    if response.status_code == 200:
        return response.json()['Results']
    return []

def get_safety_ratings_for_variant(base_url, vehicle_id):
    response = requests.get(f"{base_url}/VehicleId/{vehicle_id}")
    if response.status_code == 200:
        return response.json()['Results']
    return []

# Set base URL for NHTSA API
base_url = "https://api.nhtsa.gov/SafetyRatings"

# Target Model Year 2023
model_year = 2019

# Keep track of all safety ratings
safety_ratings_data = []

# Step 2: Get all makes for model year 2023
makes = get_all_makes_for_model_year(base_url, model_year)

# Initialize counter for top 20 records
record_count = 0
max_records = 20000

# Iterate over each make
for make in makes:
    # Step 3: Get all models for the make and model year
    models = get_all_models_for_make_and_year(base_url, model_year, make)

    # Iterate over each model
    for model in models:
        # Step 4: Get vehicle variants
        variants = get_vehicle_variants(base_url, model_year, make, model)

        # Iterate over each variant
        for variant in variants:
            if record_count >= max_records:  # Check if we've reached the limit
                break

            
            vehicle_id = variant['VehicleId']
            # Step 5: Get safety ratings for the selected variant
            safety_ratings = get_safety_ratings_for_variant(base_url, vehicle_id)
            safety_ratings_data.append({
                'ModelYear': model_year,
                'Make': make,
                'Model': model,
                'VehicleId': vehicle_id,
                'VehicleDescription': variant['VehicleDescription'],
                'SafetyRatings': safety_ratings
            })
            record_count += 1  # Increment record count
            
            # Pause to avoid overwhelming the API (optional)
            #time.sleep(0.1)

        if record_count >= max_records:  # Check again after model loop
            break

    if record_count >= max_records:  # Check again after make loop
        break

# Convert to DataFrame for analysis
df = pd.DataFrame(safety_ratings_data)

# # Expand the dictionary into separate columns
# df_expanded = df['SafetyRatings'].apply(pd.Series)

# df_expanded_1 = df_expanded[0].apply(pd.Series)

# print(df.head(20))  # Print the top 20 records

# Save data to CSV for future use
df.to_csv('safety_ratings_2019.csv', index=False)

# df_expanded_1.to_csv('2021_expanded.csv', index=False)


In [3]:

# Expand the dictionary into separate columns
df_expanded = df['SafetyRatings'].apply(pd.Series)

df_expanded_1 = df_expanded[0].apply(pd.Series)

df_expanded_1.to_csv('2019_expanded.csv', index=False)

In [ ]:
# Expand the dictionary into separate columns
df_expanded = df['SafetyRatings'].apply(pd.Series)

df_expanded.head()

In [33]:
df_expanded_1 = df_expanded[0].apply(pd.Series)

df_expanded_1.head()

,VehiclePicture,OverallRating,OverallFrontCrashRating,FrontCrashDriversideRating,FrontCrashPassengersideRating,FrontCrashPicture,FrontCrashVideo,OverallSideCrashRating,SideCrashDriversideRating,SideCrashPassengersideRating,...,NHTSAForwardCollisionWarning,NHTSALaneDepartureWarning,ComplaintsCount,RecallsCount,InvestigationCount,ModelYear,Make,Model,VehicleDescription,VehicleId
0,https://static.nhtsa.gov/images/vehicles/13221...,5,4,5,4,https://static.nhtsa.gov/crashTest/images/2022...,https://static.nhtsa.gov/crashTest/videos/2022...,5,5,5,...,Standard,Standard,2,0,0,2022,ACURA,ILX,2022 Acura ILX 4 DR FWD,16483
1,https://static.nhtsa.gov/images/vehicles/15123...,Not Rated,4,4,4,https://static.nhtsa.gov/crashTest/images/2022...,https://static.nhtsa.gov/crashTest/videos/2022...,5,5,5,...,Standard,Standard,47,1,0,2022,ACURA,MDX,2022 Acura MDX SUV AWD,15925
2,https://static.nhtsa.gov/images/vehicles/15123...,5,4,4,4,https://static.nhtsa.gov/crashTest/images/2022...,https://static.nhtsa.gov/crashTest/videos/2022...,5,5,5,...,Standard,Standard,47,1,0,2022,ACURA,MDX,2022 Acura MDX SUV FWD,15924
3,NaN,Not Rated,Not Rated,Not Rated,Not Rated,NaN,NaN,Not Rated,Not Rated,Not Rated,...,No,No,0,0,0,2022,ACURA,NSX,2022 Acura NSX 2 DR AWD,17013
4,NaN,5,4,4,4,https://static.nhtsa.gov/crashTest/images/2022...,https://static.nhtsa.gov/crashTest/videos/2022...,5,5,5,...,Standard,Standard,16,1,0,2022,ACURA,RDX,2022 Acura RDX SUV AWD,16838


In [27]:
# Import necessary library
import pandas as pd

# Sample DataFrame with a column containing dictionaries
data = {
    'VehicleData': [
        {
            'VehiclePicture': 'https://static.nhtsa.gov/images/vehicles/13221_st0640_046.png',
            'OverallRating': '5',
            'OverallFrontCrashRating': '4',
            'FrontCrashDriversideRating': '5',
            'FrontCrashPassengersideRating': '4',
            'FrontCrashPicture': 'https://static.nhtsa.gov/crashTest/images/2022/v09252P001.jpg',
            'FrontCrashVideo': 'https://static.nhtsa.gov/crashTest/videos/2022/v09252C001.wmv',
            'OverallSideCrashRating': '5',
            'SideCrashDriversideRating': '5',
            'SideCrashPassengersideRating': '5',
            'SideCrashPicture': 'https://static.nhtsa.gov/crashTest/images/2022/v09254P104.jpg',
            'SideCrashVideo': 'https://static.nhtsa.gov/crashTest/videos/2022/v09254C012.wmv',
            'combinedSideBarrierAndPoleRating-Front': '5',
            'combinedSideBarrierAndPoleRating-Rear': '5',
            'sideBarrierRating-Overall': '5',
            'RolloverRating': '4',
            'RolloverRating2': 'Not Rated',
            'RolloverPossibility': 0.107,
            'RolloverPossibility2': 0.0,
            'dynamicTipResult': 'No Tip',
            'SidePoleCrashRating': '5',
            'SidePolePicture': 'https://static.nhtsa.gov/crashTest/images/2022/v09253P072.jpg',
            'SidePoleVideo': 'https://static.nhtsa.gov/crashTest/videos/2022/v09253C012.wmv',
            'NHTSAElectronicStabilityControl': 'Standard',
            'NHTSAForwardCollisionWarning': 'Standard',
            'NHTSALaneDepartureWarning': 'Standard',
            'ComplaintsCount': 2,
            'RecallsCount': 0,
            'InvestigationCount': 0,
            'ModelYear': 2022,
            'Make': 'ACURA',
            'Model': 'ILX',
            'VehicleDescription': '2022 Acura ILX 4 DR FWD',
            'VehicleId': 16483
        }
    ]
}

# Create the DataFrame
df = pd.DataFrame(data)

print(df)

# Expand the dictionary into separate columns
df_expanded = df['VehicleData'].apply(pd.Series)

df_expanded.head()



                                         VehicleData
0  {'VehiclePicture': 'https://static.nhtsa.gov/i...


,VehiclePicture,OverallRating,OverallFrontCrashRating,FrontCrashDriversideRating,FrontCrashPassengersideRating,FrontCrashPicture,FrontCrashVideo,OverallSideCrashRating,SideCrashDriversideRating,SideCrashPassengersideRating,...,NHTSAForwardCollisionWarning,NHTSALaneDepartureWarning,ComplaintsCount,RecallsCount,InvestigationCount,ModelYear,Make,Model,VehicleDescription,VehicleId
0,https://static.nhtsa.gov/images/vehicles/13221...,5,4,5,4,https://static.nhtsa.gov/crashTest/images/2022...,https://static.nhtsa.gov/crashTest/videos/2022...,5,5,5,...,Standard,Standard,2,0,0,2022,ACURA,ILX,2022 Acura ILX 4 DR FWD,16483
